In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split 

In [34]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [35]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [36]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [37]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(df.drop(columns = ['has_covid']),
                                                df['has_covid'],test_size=0.2,
                                                random_state=0)
X_train.shape,X_test.shape

((80, 5), (20, 5))

In [21]:
#Add simple imputer on fever col
ls=SimpleImputer()
#fit and transform on traon and test data 
X_train_fiver = ls.fit_transform(X_train[['fever']])

X_test_fiver = ls.fit_transform(X_test[['fever']])
X_train_fiver.shape

(80, 1)

In [22]:
#Add ordinal encoding on cough col
oe=OrdinalEncoder(categories=[['Mild','Strong']])
#fit and transform on traon and test data 
X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [23]:
ohe = OneHotEncoder(drop='first',sparse=False)   #if we use sparce then we not use .toarray()

In [24]:
# X_train_new = ohe.fit_transform(X_train[['fuel','owner']]).toarray()
X_train_city = ohe.fit_transform(X_train[['gender','city']])
X_test_city = ohe.transform(X_test[['gender','city']])

C:\Users\pc\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:
X_train_city.shape

(80, 4)

In [26]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city'])
X_test_age = X_test.drop(columns=['gender','fever','cough','city'])

In [27]:
X_train_concate = np.concatenate((X_train_age,X_train_fiver,X_train_city,X_train_cough),axis=1)
X_test_concate = np.concatenate((X_test_age,X_test_fiver,X_test_city,X_test_cough),axis=1)

In [29]:
X_train_concate.shape

(80, 7)

# Transforming concept used 

In [39]:
from sklearn.compose import ColumnTransformer

In [40]:
ct = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [42]:
ct.fit_transform(X_train).shape

C:\Users\pc\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [43]:
ct.fit_transform(X_test).shape

C:\Users\pc\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)